# Sam Controlnet Train
This notebook contains code to train our custom controlnet based on the sam-controlnet-dataset

## Imports

In [1]:
import os

if os.environ.get('KAGGLE_KERNEL_RUN_TYPE', ''):
    !pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
    !pip install git+https://github.com/huggingface/diffusers.git wandb 
    !pip install -q transformers==4.29.0
    !pip install -q accelerate==0.19.0
    !pip install -q huggingface-hub==0.14.1
    !pip install -q datasets==2.12.0
    !pip install -q xformers==0.0.16
    # !pip install -q git+https://github.com/huggingface/diffusers you can do that instead of next 3 lines no???
    !git clone https://github.com/huggingface/diffusers
    %cd diffusers
    !pip install -e .

Looking in indexes: https://download.pytorch.org/whl/cu118
  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-iu5_4l92
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-iu5_4l92
  Resolved https://github.com/huggingface/diffusers.git to commit cd9d0913d90b52d3609427ea2ad5d07b557677dd
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - done
  Preparing metadata (pyproject.toml) ... - done
  Created wheel for diffusers: filename=diffusers-0.17.0.dev0-py3-none-any.whl size=1081832 sha256=160d59db4337129f61c66cc0cfb82fcb6777df20aad10b862769be24091b16a2
  Stored in directory: /tmp/pip-ephem-wheel-cache-jorcw3xh/wheels/4d/b7/a8/6f9549ceec5daad78675b857ac57d697c387062506520a7b50
Successfully built diffusers
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the

In [2]:
!git clone https://github.com/huggingface/diffusers
%cd diffusers
!pip install -e .

Cloning into 'diffusers'...
remote: Enumerating objects: 26796, done.
remote: Counting objects: 100% (360/360), done.
remote: Compressing objects: 100% (252/252), done.
remote: Total 26796 (delta 206), reused 195 (delta 89), pack-reused 26436
Receiving objects: 100% (26796/26796), 18.36 MiB | 21.12 MiB/s, done.
Resolving deltas: 100% (19494/19494), done.
/kaggle/working/diffusers/diffusers
Obtaining file:///kaggle/working/diffusers/diffusers
  Installing build dependencies ... - \ | done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... - done
  Preparing editable metadata (pyproject.toml) ... - done
  Building editable for diffusers (pyproject.toml) ... - done
  Created wheel for diffusers: filename=diffusers-0.17.0.dev0-0.editable-py3-none-any.whl size=10525 sha256=048b367fde1300902aabb3ec948ee0dc3fd0cfe167e1c95f5add787e3aee40c2
  Stored in directory: /tmp/pip-ephem-wheel-cache-85cvorce/wheels/b6/4d/b3/0f56ce2a7ce37a

In [3]:
%cd examples/controlnet

/kaggle/working/diffusers/diffusers/examples/controlnet


In [4]:
!ls
!pip install -r requirements.txt

README.md	  requirements_flax.txt  train_controlnet_flax.py
requirements.txt  train_controlnet.py
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.4 MB/s eta 0:00:00


In [5]:
!wget https://datasets-server.huggingface.co/assets/baptistecolle/sam-controlnet-final/--/baptistecolle--sam-controlnet-final/train/0/conditioning_image/image.jpg
!mv image.jpg image0.jpg
!wget https://datasets-server.huggingface.co/assets/baptistecolle/sam-controlnet-final/--/baptistecolle--sam-controlnet-final/train/1/conditioning_image/image.jpg
!mv image.jpg image1.jpg

--2023-06-07 14:55:42--  https://datasets-server.huggingface.co/assets/baptistecolle/sam-controlnet-final/--/baptistecolle--sam-controlnet-final/train/0/conditioning_image/image.jpg
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 34.200.186.24, 50.17.173.235, 44.197.252.161, ...
Connecting to datasets-server.huggingface.co (datasets-server.huggingface.co)|34.200.186.24|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21234 (21K) [image/jpeg]
Saving to: ‘image.jpg’

image.jpg           100%[===================>]  20.74K  --.-KB/s    in 0.06s   

2023-06-07 14:55:43 (327 KB/s) - ‘image.jpg’ saved [21234/21234]

--2023-06-07 14:55:45--  https://datasets-server.huggingface.co/assets/baptistecolle/sam-controlnet-final/--/baptistecolle--sam-controlnet-final/train/1/conditioning_image/image.jpg
Resolving datasets-server.huggingface.co (datasets-server.huggingface.co)... 34.200.186.24, 50.17.173.235, 44.197.252.161, ...
Connecting to datase

In [6]:
!mkdir model

In [7]:
!accelerate config default
!mv /root/.cache/huggingface/accelerate/default_config.yaml /content/
!mv /content/default_config.yaml /root/.cache/huggingface/accelerate/
!accelerate config update

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml
mv: cannot move '/root/.cache/huggingface/accelerate/default_config.yaml' to '/content/': Not a directory
mv: cannot stat '/content/default_config.yaml': No such file or directory
Sucessfully updated the configuration file at /root/.cache/huggingface/accelerate/default_config.yaml.


In [8]:
!wandb login $WANDB_API_KEY

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## Training

In [9]:
%env MODEL_DIR=runwayml/stable-diffusion-v1-5
%env OUTPUT_DIR=model/

!accelerate launch train_controlnet.py \
 --pretrained_model_name_or_path=runwayml/stable-diffusion-v1-5 \
 --controlnet_model_name_or_path=lllyasviel/sd-controlnet-seg \
 --output_dir=$OUTPUT_DIR \
 --dataset_name=baptistecolle/sam-controlnet-final \
 --resolution=512 \
 --learning_rate=1e-5 \
 --validation_image "./image0.png" "./image1.png" \
 --validation_prompt "A woman wearing a net on her head cutting a cake. " "A young boy standing in front of a computer keyboard." \
 --train_batch_size=1 \
 --gradient_accumulation_steps=4 \
 --gradient_checkpointing \
 --use_8bit_adam \
 --enable_xformers_memory_efficient_attention \
 --set_grads_to_none 
 --image_column=image \
 --caption_column=text \
 --report_to=wandb \
 --resume_from_checkpoint

IndentationError: unexpected indent (1136308320.py, line 5)